In [0]:
import pandas as pd
import numpy as np
import sklearn

np.random.seed(42)

In [2]:
# from google.colab import files
# files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving submission_rnd.csv to submission_rnd.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


{'sample_submission.csv': b'example_id,label\n24bf52cc84b75bce8ee9c0cdd0b5117f,False\n708cc462eca0c274bcff304bbc4396f8,False\nc5da1d371af9525d2fc2174311e3dae4,False\n844f5f5be464a0600891ac6b8c1a1afd,False\n97daba48c36db68f3e84ebcbf8952aa2,True\n5aa7363096040ffc17b20f595edda878,False\na096ea460d0c9f2af7211c8f2d1d7ee7,True\nff4a30ad03aa3b4485a542dd32919722,True\n0841f6a99ed3dd2ba9b8cec3b0a1451c,True\n6d368e9b25aebb7bfbda01c4eb7533f4,False\n42ab4246d252a4d7ca12d5a0de05510d,False\n7b2c87b901e417e67cad977d68c24b75,False\n0265e736477ce905ce806b6f22e61ede,True\n2fe99aa2e46d60c61635270ffb9f54ac,True\n2af25f5c41f8dd2f5ad7d1add76b2d88,True\nd6da85368b9ef51e810cdddd45f0a025,False\n5eff6423e3fa1c8cfe4b258cc37e9dee,True\n32373b8de1abf9a7bb2a44f4b56dff9e,False\na5da7a0f62a1aea843b76559aa027669,True\ncb33bca5c4d0f54ee1d3e3589c9ad3c0,True\nd8f8bb8624af49d958a72ac1752d9b5c,False\n1131c61c63084a8f8461ec4eb4e9a857,True\nd5018910d41f9a955e29c6e9eaf4e950,False\nc752e2a9c8d538a23977b04bf36ddffe,False\n5d93f

In [3]:
df = pd.read_csv('train.csv')
df.head()

,example_id,text,label
0,140d03eabb7cb5c2558605eb8336689c,brandpost best of both worlds hybrid onsite an...,0
1,f7f1f906c9e2b76e63020f8794516185,$mention$ they shall in all cases except treas...,0
2,39f0b2ebc12e008a7a43ec318d0c3874,lifelock offers to protect you from the equifa...,0
3,ad4e57c69f00548253cb6d47b15c3ce4,skimmer adware spent two months in google play...,1
4,236bfe8f2f145dbcf17be122546946db,just want to love and be loved,0


In [0]:
import spacy

nlp = spacy.load("en_core_web_sm", disable=["tagger", "parser", "ner"])

In [0]:
df['cleaned'] = df['text'].apply(lambda x: [t.lemma_ for t in nlp(x.replace('\n', ' ').replace('\r', ' ')) if not (t.is_stop or t.is_punct or t.text[0]==' ')])

In [6]:
df['cleaned'][10]

['New', 'Mac', 'malware', 'loose', 'need', 'know', '$', 'URL$', '@$URL$']

In [11]:
def flat_nested(nested):
    flatten = []
    for item in nested:
        if isinstance(item, list):
            flatten.extend(item)
        else:
            flatten.append(item)
    return flatten

vocab = set(flat_nested(df.cleaned.tolist()))

len(vocab)

3886

In [10]:
vocabulary = {}

for i, token in enumerate(vocab): 
    vocabulary[token] = i

vocabulary['available']

1930

In [0]:
vectors = np.zeros((df.shape[0], len(vocab)), dtype=np.int16)
for i in range(vectors.shape[0]):
    for token in df['cleaned'][i]: 
        try: 
            vectors[i][vocabulary[token]] += 1
        except KeyError: 
            continue

In [0]:
from sklearn.model_selection import train_test_split

train_vec, test_vec, train_lab, test_lab = train_test_split(vectors, df['label'], test_size=0.4, random_state=42)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

cls = SVC().fit(train_vec, train_lab)
P = cls.predict(test_vec)

P

array([0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,

In [32]:
PP = (test_lab*P).sum().sum()
PN = (test_lab*(1-P)).sum().sum()
NP = ((1-test_lab)*P).sum().sum()
NN = ((1-test_lab)*(1-P)).sum().sum()
T = P.size


TP = PP/T 
FP = NP/T   
FN = PN/T 
TN = NN/T  


prec = TP/(TP+FP)  ## Your code here 
rec = TP/(TP+FN)  ## Your code here 
F1 = 2*prec*rec/(prec+rec)  ## Your code here 
F2 = 5*prec*rec/(4*prec+rec) ## Your code here 

print(TP, TN, FP, FN)
print(prec, rec)
print(F1, F2)

0.3277310924369748 0.32142857142857145 0.2710084033613445 0.07983193277310924
0.5473684210526316 0.8041237113402062
0.651356993736952 0.7351555136663525


In [36]:
df = pd.read_csv('test.csv')
df['cleaned'] = df['text'].apply(lambda x: [t.lemma_ for t in nlp(x.replace('\n', ' ').replace('\r', ' ')) if not (t.is_stop or t.is_punct or t.text[0]==' ')])

vectors = np.zeros((df.shape[0], len(vocab)), dtype=np.int16)
for i in range(vectors.shape[0]):
    for token in df['cleaned'][i]: 
        try: 
            vectors[i][vocabulary[token]] += 1
        except KeyError: 
            continue

P = cls.predict(vectors)
print(P)
df['label'] = P
df.head()

df[['example_id', 'label']].to_csv('subm.csv')

[0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0 1 0
 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 1
 0 1 0 0 0 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 0 0 0 0
 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1
 1 0 0 0 1 1 0 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 0 0 0
 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 1 1
 0 1 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0
 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0
 1 0 0 0 0 0 1 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1
 0 0 1 0 0 0 1 0 1 1 1 1 